# UNUSED

# Importing Libraries


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import helper
from helper import *

In [ ]:
import importlib
importlib.reload(helper)


# Data Collection and cleaning

In [ ]:
# basic variables

start_date = '2000-01-01'
end_date = '2023-12-31' 
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

In [ ]:
symbols = collect_tickers('US', start_date, end_date)

In [ ]:
snp_stocks_data = download_data(symbols, start_date, end_date)

In [ ]:
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)

In [ ]:
symbols = list(snp_stocks_data.keys())

In [ ]:
len(symbols)

In [ ]:
returns_df = calculate_return(symbols, snp_stocks_data, end_date, rebalance='W')

In [ ]:
returns_df

# Benchmark Data

In [ ]:
data = benchmark_data('US', start_date, end_date)

In [ ]:
price = data.resample('ME').last()

In [ ]:
gains = benchmark_returns(data)
gains.index = pd.to_datetime(gains.index, format='%Y-%m-%d')

# Strategies

In [ ]:
'''
A dataframe to store the return%. 
The strategy name is the row name and the timestamp is column name
'''
strategy_returns = pd.DataFrame(columns=pd.date_range(start=start_date, end=end_date, freq='ME'))

## Momentum strategy

In [ ]:
def momentum_long(snp_stocks_data, momentum_df, returns_df, gains,p,strategy='', strategy_returns=strategy_returns):
    # Stocks in the top p-percentile 
    top_p_momentum_stocks_dict = top_p_percentage_stocks(momentum_df, p)

    # Returns from longing the top p-percentile
    long_dict = total_return(snp_stocks_data, top_p_momentum_stocks_dict, returns_df, 20)
    long = pd.DataFrame.from_dict(long_dict, orient='index')
    long.rename({0:strategy}, axis='columns', inplace=True)
    strategy_returns[strategy] = long
    x, plot = fig_plot(long_dict, gains, f'{p}% long')
    return x, plot

 # Go long on top p-percentile and sort on bottom p-percentile
def momentum_long_sort(snp_stocks_data, momentum_df, returns_df,gains, p_long, p_sort, strategy='', strategy_returns=strategy_returns):
    # Stocks in the top p-percentile 
    top_p_momentum_stocks_dict = top_p_percentage_stocks(momentum_df, p_long)


    # Stocks in the bottom p-percentile
    bottom_p_momentum_stocks_dict = bottom_p_percentage_stocks(momentum_df, p_sort)
  
  
    # Retuens from longing as well as sorting at the same time
    sort_long_dict = long_sort_return(snp_stocks_data, top_p_momentum_stocks_dict, bottom_p_momentum_stocks_dict, returns_df)
    long = pd.DataFrame.from_dict(sort_long_dict, orient='index')
    long.rename({0:strategy}, axis='columns')
    strategy_returns[strategy] = long

    x, plot = fig_plot(sort_long_dict, gains, f'{p_long}% long and {p_sort}% sort')

    return x, plot

In [ ]:
def momentum_12_1(momentum_12_1_df, p, strategy_returns=strategy_returns):
    # Stocks from the top p-percentile
    top_p_momentum_stocks_dict_12_1 = top_p_percentile_stocks(momentum_12_1_df, p)

    # Returns from the top p-percentile
    long_dict_12_1 = total_return(snp_stocks_data, top_p_momentum_stocks_dict_12_1, returns_df)
    long = pd.DataFrame.from_dict(long_dict_12_1, orient='index')
    long.rename({0:f'12_1_momentum_{p}'}, axis='columns')
    strategy_returns[strategy] = long

    x, plot = fig_plot(long_dict_12_1, gains)
    return x, plot

def momentum_12_1_ls(snp_stocks_data, momentum_12_1_df, returns_df, gains, p_top, p_bottom, strategy_returns=strategy_returns):
    # Stocks from the top p-percentile
    top_p_momentum_stocks_dict_12_1 = top_p_percentile_stocks(momentum_12_1_df, p_top)
    # Stocks in the bottom p-percentile
    bottom_p_momentum_stocks_dict_12_1 = bottom_p_percentile_stocks(momentum_12_1_df, p_bottom)
    # Returns from longing as well as sorting at the same time
    sort_long_dict_12_1 = long_sort_return(snp_stocks_data, top_p_momentum_stocks_dict_12_1, bottom_p_momentum_stocks_dict_12_1, returns_df)
    long = pd.DataFrame.from_dict(sort_long_dict_12_1, orient='index')
    long.rename({0:f'12_1_momentum_ls_{p_top}_{p_bottom}'}, axis='columns')
    strategy_returns[strategy] = long

    x, plot = fig_plot(sort_long_dict_12_1, gains)
    return x, plot

In [ ]:
# Buy if price above m-month MA
def momentum_abv_ma(snp_stocks_data, momentum_df, returns_df, gains,p, strategy_returns=strategy_returns):
    # Stocks in the top p-percentile 
    top_p_momentum_stocks_dict = top_p_percentile_stocks(momentum_df, p)
    momentum_6_ma = stocks_moving_average(momentum_dict=top_p_momentum_stocks_dict, m= 6 ,stock_data_dict=snp_stocks_data)

    # Returns from longing the top p-percentile
    above_ma_returns = total_return(snp_stocks_data, momentum_6_ma, returns_df)
    long = pd.DataFrame.from_dict(above_ma_returns, orient='index')
    long.rename({0:f'momentum_above_ma_{p}'}, axis='columns')
    strategy_returns[strategy] = long

    x, plot = fig_plot(above_ma_returns, gains, f'{p}% long')
    return x, plot


## Reversion Strategy

In [ ]:
def reversion_long(snp_stocks_data, momentum, returns_df, gains, p, strategy='', strategy_returns=strategy_returns):
    # Stocks in the top p-percentile 
    top_p_momentum_stocks_dict_inversion = top_p_percentile_stocks(momentum, p)

    # Returns from longing the top p-percentile
    long_dict_reversion = total_return(snp_stocks_data, top_p_momentum_stocks_dict_inversion, returns_df)
    long = pd.DataFrame.from_dict(long_dict_reversion, orient='index')
    long.rename({0:strategy}, axis='columns')
    strategy_returns[strategy] = long
    x, plot = fig_plot(long_dict_reversion, gains)
    return x, plot

def reversion_long_sort(snp_stocks_data, momentum, returns_df, gains, p_long, p_sort, strategy=''):
    # Stocks in the top p-percentile 
    top_p_momentum_stocks_dict_inversion = top_p_percentile_stocks(momentum, p_long)

    # Stocks in the bottom p-percentile
    bottom_p_momentum_stocks_dict_inversion = bottom_p_percentile_stocks(momentum, p_sort)

    # Retuens from longing as well as sorting at the same time
    sort_long_dict_reversion = long_sort_return(snp_stocks_data, top_p_momentum_stocks_dict_inversion, bottom_p_momentum_stocks_dict_inversion, returns_df)
    long = pd.DataFrame.from_dict(sort_long_dict_reversion, orient='index')
    long.rename({0:strategy}, axis='columns')
    strategy_returns[strategy] = long
    x, plot = fig_plot(sort_long_dict_reversion, gains)
    return x, plot


## Value strategy

In [ ]:
def price_strategy(stocks_data, returns_df, gains, p_greater, p_less, strategy='', strategy_returns=strategy_returns):
    x = value_strategy(stocks_data, p_greater, p_less)
    m = total_return(stocks_data, x, returns_df)
    long = pd.DataFrame.from_dict(m, orient='index')
    long.rename({0:strategy}, axis='columns')
    strategy_returns[strategy] = long
    a, plot = fig_plot(m, gains)
    return a, plot

## Seasonality strategy

In [ ]:
# If stock changed more than p%, we buy(sell)
def yearly_seasonality(snp_stocks_data, returns_df, gains, p, strategy1, strategy2, strategy_returns=strategy_returns):
    long, sort = seas_prev_mth(snp_stocks_data, p)
    long_ret = total_return(snp_stocks_data, long, returns_df)
    long_sort_returns = long_sort_return(snp_stocks_data, long, sort, returns_df)
    long = pd.DataFrame.from_dict(long_sort_returns, orient='index')
    long.rename({0:strategy2}, axis='columns')
    strategy_returns[strategy2] = long
    long = pd.DataFrame.from_dict(long_ret, orient='index')
    long.rename({0:strategy1}, axis='columns')
    strategy_returns[strategy1] = long
    x,x_plot = fig_plot(long_ret, gains)
    y,y_plot = fig_plot(long_sort_returns, gains)
    return x, x_plot, y, y_plot

In [ ]:
def sell_may():
    pass

### Volatility

In [ ]:
def volatility_strategy(stocks_data, returns_df, m, p, strategy):
    vol_df = calculate_m_month_volatility(stocks_data, m)
    top = top_p_percentage_stocks(vol_df, p)
    long_dict_reversion = total_return(snp_stocks_data, top, returns_df)
    long = pd.DataFrame.from_dict(long_dict_reversion, orient='index')
    long.rename({0:strategy}, axis='columns')
    strategy_returns[strategy] = long
    x, plot = fig_plot(long_dict_reversion, gains)
    return x

def volatility_strategy_ls(stocks_data, returns_df, m, p, strategy):
    vol_df = calculate_m_month_volatility(stocks_data, m)
    top = top_p_percentage_stocks(vol_df, p)
    btm = top_p_percentage_stocks(vol_df, p)
    sort_long_dict_reversion = long_sort_return(snp_stocks_data, top, btm, returns_df)
    long = pd.DataFrame.from_dict(sort_long_dict_reversion, orient='index')
    long.rename({0:strategy}, axis='columns')
    strategy_returns[strategy] = long
    x, plot = fig_plot(sort_long_dict_reversion, gains)
    return x, plot

# Plots

In [ ]:
plots = {}
sharpe_ratios = {}
pct = [10, 20, 30]
months = [3, 6, 9]

## Momentum Strategy

### Going long only for the top p% stocks

In [ ]:
for m in months:
    print(f'{m}-month momentum')
    momentum_data = {}
    for symbol, data in snp_stocks_data.items():
        momentum_data[symbol] = calculate_m_momentum(data, m)

    # Convert the momentum data dictionary to a DataFrame
    momentum = pd.DataFrame.from_dict(momentum_data, orient='index')
    
    
    for p in pct:
        strategy = f'{m}_momentum_{p}'
        x , plot_m= momentum_long(snp_stocks_data, momentum, returns_df, gains, p, strategy= strategy)

### Going long on top p% and short on bottom p%

In [ ]:
for m in months:
    print(f'{m}-month momentum')
    momentum_data = {}
    for symbol, data in snp_stocks_data.items():
        momentum_data[symbol] = calculate_m_momentum(data, m)

    for p in pct:
        strategy = f'{m}_momentum_ls_{p}'
        x, plot_m = momentum_long_sort(snp_stocks_data, momentum, returns_df, gains, p, p, strategy)
        plots[f'{m}-month_momentum_ls{p}'] = plot_m
        sharpe_ratios[f'{m}-month_momentum_ls{p}'] = x
        


### 12-1 momentum strategy

In [ ]:
momentum_data = {}
for symbol, data in snp_stocks_data.items():
    momentum_data[symbol] = calculate_12_1_momentum(data)

    # Convert the momentum data dictionary to a DataFrame
momentum_12_1_df = pd.DataFrame.from_dict(momentum_data, orient='index')
for p in pct:
    x, plot_m = momentum_12_1(momentum_12_1_df, p)
    x

### 12-1 long-sort

In [ ]:
for p_top in pct:
    x, plot_m = momentum_12_1_ls(snp_stocks_data, momentum_12_1_df, returns_df, gains, p_top, p_top)
    x

### Buying only if the current price is above the 6-month MA

In [ ]:
# pct = [10, 20, 30]
# for m in months:
#     print(f'{m}-month momentum')
#     momentum_data = {}
#     for symbol, data in snp_stocks_data.items():
#         momentum_data[symbol] = calculate_m_momentum(data, m)

#     # Convert the momentum data dictionary to a DataFrame
#     momentum = pd.DataFrame.from_dict(momentum_data, orient='index')
     
#     for p in pct:
#         strategy = f'{m}_month momentum_abv_6ma_{p}'
#         x, plot_m = momentum_abv_ma(snp_stocks_data, momentum, returns_df, gains, p)
#         plots[f'{m}_month momentum_abv_6ma'] = plot_m
#         x

## Reversion Strategy

#### long the bottom stocks

In [ ]:
for m in range(1, 36, 60):
    print(f'{m}-months')
    momentum_data = {}
    for symbol, data in snp_stocks_data.items():
        momentum_data[symbol] = calculate_m_momentum(data, m)

    # Convert the momentum data dictionary to a DataFrame
    momentum_reversion = pd.DataFrame.from_dict(momentum_data, orient='index')

    for p in pct:
        strategy = f'{m}-reversal_{p}'
        print(p)
        x, plot_m = reversion_long(snp_stocks_data, momentum_reversion, returns_df, gains, p)
        plots[f'{m}_month_reversion'] = plot_m
        x

#### Long on bottom stocks and short on top stocks

In [ ]:
for m in range(1, 4):
    print(f'{m}-months')
    momentum_data = {}
    for symbol, data in snp_stocks_data.items():
        momentum_data[symbol] = calculate_m_momentum(data, 3)

    # Convert the momentum data dictionary to a DataFrame
    momentum_reversion = pd.DataFrame.from_dict(momentum_data, orient='index')

    for p_long in pct:
        for p_sort in pct:
            strategy = f'{m}-reversal_ls_{p}'
            x, plot_m = reversion_long_sort(snp_stocks_data, momentum_reversion, returns_df, gains, p_long, p_sort)
            plots[f'{m}_month_reversion'] = plot_m
            x

## Seasonality

#### Buy from November to May

In [ ]:
# # Ensure both DataFrames have the same index for correct comparison
# strategy_gains = trade_returns_seasonality.reindex(gains.index).fillna(0)

# # Calculate cumulative returns
# cumulative_benchmark_returns = (1 + gains / 100).cumprod() - 1
# seasonality_cumulative_momentum_returns = (1 + strategy_gains['Trade Returns'] / 100).cumprod() - 1


In [ ]:

# # Create a Plotly figure
# fig = go.Figure()

# # Add the cumulative benchmark returns line trace
# fig.add_trace(
#     go.Scatter(
#         x=cumulative_benchmark_returns.index,
#         y=cumulative_benchmark_returns,
#         mode='lines',
#         name='Cumulative Benchmark Returns'
#     )
# )

# # Add the cumulative momentum returns line trace
# fig.add_trace(
#     go.Scatter(
#         x=seasonality_cumulative_momentum_returns.index,
#         y=seasonality_cumulative_momentum_returns,
#         mode='lines',
#         name='Seasonality Returns'
#     )
# )

# # Configure the x-axis to show months when zoomed in and add range selector buttons
# fig.update_layout(
#     title='Cumulative Returns: Seasonality vs Benchmark from 01-01-2001',
#     xaxis_title='Date',
#     yaxis_title='Cumulative Return %',
#     xaxis=dict(
#         tickmode='auto',
#         rangeslider=dict(
#             visible=True
#         ),
#         rangeselector=dict(
#             buttons=list([
#                 dict(count=1, label="1m", step="month", stepmode="backward"),
#                 dict(count=6, label="6m", step="month", stepmode="backward"),
#                 dict(count=1, label="YTD", step="year", stepmode="todate"),
#                 dict(count=1, label="1y", step="year", stepmode="backward"),
#                 dict(step="all")
#             ])
#         ),
#         type="date"
#     ),
# )

# # Show the plot
# fig.show()

In [ ]:
# # Calculate the yearly stats
# returns_dict = strategy_gains['Trade Returns'].to_dict()
# yearly_stats_df = calculate_yearly_stats(returns_dict, gains)

# yearly_stats_df.T

#### Buy only if stock increased > p% last year same month

In [ ]:
pct_above = [10, 15]
seasonality={}

for p in pct_above:
    strategy1 = f'seasonality_above{p}'
    strategy2 = f'seasonality_above{p}_ls'
    print(p)
    x, plot_x, y, plot_y =yearly_seasonality(snp_stocks_data,  returns_df, gains, p, strategy1, strategy2)
    seasonality[f'{p}% buy'] = x
    seasonality[f'{p}% buy_sell'] = y
    seasonality[f'{p}% buy_plot'] = plot_x
    seasonality[f'{p}% buy_sell_plot'] = plot_y


## Value Strategy

In [ ]:
pct= [ 25, 50, 75]
for p in pct:
    print(p)
    x = price_strategy(snp_stocks_data, returns_df, gains, p)

______________________________________________________________________________________________________________________________________________________

# Stats

In [ ]:
def combine_strategy_returns(strategy_returns_dict):
    # Create an empty list to store DataFrames
    dfs = []
    
    # Iterate over the dictionary
    for strategy, df in strategy_returns_dict.items():
        # Rename the column to the strategy name
        df = df.rename(columns={df.columns[0]: strategy})
        # Append the DataFrame to the list
        dfs.append(df)
    
    # Concatenate all DataFrames in the list along the columns
    combined_df = pd.concat(dfs, axis=1)
    
    return combined_df

In [ ]:
strategy_returns_df = combine_strategy_returns(strategy_returns)

In [ ]:
y=10
y_years_cagr(pd.DataFrame(gains), y)

In [ ]:
y = 1
sharpe_ratio_df = y_years_sharpe(strategy_returns_df, y)
sharpe_ratio_df

# ML

# NIFTY 500

In [ ]:
nse_tickers = pd.read_html('https://ournifty.com/stock-list-in-nse-fo-futures-and-options.html#:~:text=NSE%20F%26O%20Stock%20List%3A%20%20%20%20SL,%20%201000%20%2052%20more%20rows%20')[0]

In [ ]:

nse_tickers = nse_tickers.SYMBOL.to_list()

In [ ]:
len(nse_tickers)

In [ ]:
for count in range(len(nse_tickers)):
    nse_tickers[count] = nse_tickers[count] + ".NS"

In [ ]:
price_data = yf.download(nse_tickers , start = '2021-01-01', end = '2021-08-25')

In [ ]:
price_data

In [ ]:
nse_stocks_data = {}

In [ ]:
for i in symbols[:10]:
    nse_stocks_data[i] = yf.download(i, start_date, end_date)

In [ ]:
nse_stocks_data

________________________________________________________________________________________

# Extras

In [ ]:

def calculate_return_signals(return_df, y):
    # Create a dictionary to store the results
    results_dict = {stock: {} for stock in return_df.index}

    for current_date in return_df.columns:
        for stock in return_df.index:
            
            signals = []
            for i in range(1, y + 1):
                
                past_date = current_date - pd.DateOffset(weeks=52*i)
                if past_date in return_df.columns:
                    print(past_date)
                    past_return = return_df.at[stock, past_date]
                    signal = 1 if past_return > 0 else -1
                    signals.append(signal)
                    print(signal)

            # Store the signals if we have y years of data
            results_dict[stock][current_date] = signals

    # Convert the results dictionary to a DataFrame
    results_df = pd.DataFrame.from_dict(results_dict, orient='index')

    return results_df

In [ ]:
x = calculate_return_signals(returns_df, y=3)

In [ ]:
def generate_buy_sell_signals(signals_df, m, n):
    buy_dict = {}
    sell_dict = {}

    # Iterate through each date (timestamp)
    for date in signals_df.columns:
        buy_stocks = []
        sell_stocks = []

        # Iterate through each stock in the DataFrame
        for stock in signals_df.index:
            if signals_df.at[stock, date].count(1) >= m:
                buy_stocks.append(stock)
            if signals_df.at[stock, date].count(-1) >= n:
                sell_stocks.append(stock)

        # Store the lists in the dictionaries if there are any stocks to buy or sell
        if buy_stocks:
            buy_dict[date] = buy_stocks
        if sell_stocks:
            sell_dict[date] = sell_stocks

    return buy_dict, sell_dict

In [ ]:
buy_dict, sell_dict = generate_buy_sell_signals(x, 2, 2)

In [ ]:
buy_dict

In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

In [ ]:
def plot_strategy_performance(combined_df, y):
    # Calculate cumulative returns
    cumulative_returns_df = combined_df.apply(lambda x: np.cumprod(1 + x / 100) - 1, axis=0)

    # Set the plotting style
    sns.set(style="whitegrid")

    # Create a plot for each strategy
    for strategy in combined_df.columns:
        plt.figure(figsize=(14, 7))
        
        # Calculate Sharpe ratio and CAGR
        strategy_returns = combined_df[strategy].dropna()
        sharpe_ratio = y_years_sharpe(strategy_returns, y)
        cagr = y_years_cagr(strategy_returns, y)
        
        # Plot cumulative returns
        plt.plot(cumulative_returns_df.index, cumulative_returns_df[strategy], label='Cumulative Return')
        
        # Add annotations
        plt.title(f'{strategy} Performance')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return')
        plt.legend(loc='best')
        plt.figtext(0.15, 0.85, f'Sharpe Ratio (last {y} years): {sharpe_ratio:.2f}', fontsize=12)
        plt.figtext(0.15, 0.80, f'CAGR (last {y} years): {cagr:.2f}%', fontsize=12)
        
        # Show the plot
        plt.show()

In [ ]:
plot_strategy_performance(strategy_returns_df, y=5)

In [ ]:
pct_change()

In [ ]:

# # Example function to generate individual traces (replace with your actual data)
# def generate_trace(name):
#     return go.Scatter(x=[1, 2, 3], y=[10, 11, 12], mode='lines', name=name)
# Create a single Plotly figure
fig = go.Figure()

# Add each individual trace to the figure
for title, trace in plots.items():
    fig.add_trace(trace)

# Update layout to adjust titles and overall layout
fig.update_layout(
    title_text="Overlay of Different Plots",
    xaxis_title="X-axis",
    yaxis_title="Y-axis",
    legend_title="Plots",
    height=600
)

# Show the combined plot
fig.show()

_____________________________________________________________________________________________________________________

# Presentation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
momentum_data = {}
for symbol, data in snp_stocks_data.items():
    momentum_data[symbol] = calculate_m_momentum(data, 6)

# Convert the momentum data dictionary to a DataFrame
momentums = pd.DataFrame.from_dict(momentum_data, orient='index')

In [ ]:
x = top_p_percentage_stocks(momentums, 33)

In [ ]:
y_mom = total_return(snp_stocks_data, x, returns_df)

In [ ]:
import matplotlib.dates as mdates

def plot_cumulative_returns(returns_dict, title):
    
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(list(returns_dict.items()), columns=['Timestamp', 'Return'])

    # Convert the Timestamp column to datetime
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # Sort the DataFrame by Timestamp
    df = df.sort_values(by='Timestamp')

    # Calculate the cumulative returns
    df['Cumulative Return'] = (1 + (df['Return']/100)).cumprod() - 1
    df['Cumulative Return'] *= 100

    # Plot the cumulative returns
    plt.figure(figsize=(10, 6))
    plt.plot(df['Timestamp'], df['Cumulative Return'], linestyle='-', color='black', linewidth=2)
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Cumulative Return %')
    # Customize the axes
    ax = plt.gca()
    ax.spines['top'].set_color('none')
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(2)
    ax.spines['right'].set_color('none')
    ax.xaxis.label.set_color('black')
    ax.yaxis.label.set_color('black')
    ax.tick_params(axis='x', colors='black', width=2, rotation=45) # Rotate x-axis labels by 45 degrees
    ax.tick_params(axis='y', colors='black', width=2)
    # Make the background transparent
    ax.patch.set_alpha(0)

    # Set x-axis limits to the first and last years in the data
    plt.xlim(df['Timestamp'].iloc[0].replace(month=1, day=1), df['Timestamp'].iloc[-1].replace(month=12, day=31))
    
    # Customize the x-axis date formatter
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    plt.show()


In [ ]:
y_mom

In [ ]:
plot_cumulative_returns(y_mom, '6-month Momentum Strategy')

In [ ]:
momentum_data = {}
for symbol, data in snp_stocks_data.items():
    momentum_data[symbol] = calculate_12_1_momentum(data)
   # Convert the momentum data dictionary to a DataFrame
momentum_12_1_df = pd.DataFrame.from_dict(momentum_data, orient='index')


In [ ]:
x = top_p_percentage_stocks(momentum_12_1_df, 10)

In [ ]:
y_12 = total_return(snp_stocks_data, x, returns_df)

In [ ]:
plot_cumulative_returns(y, '12-1 momentum')

In [ ]:

for symbol, data in snp_stocks_data.items():
    momentum_data[symbol] = calculate_m_momentum(data, 60)

    # Convert the momentum data dictionary to a DataFrame
momentum_reversion = pd.DataFrame.from_dict(momentum_data, orient='index')


In [ ]:
x = top_p_percentage_stocks(momentum_reversion, 33)

In [ ]:
y_rev = total_return(snp_stocks_data, x,returns_df)

In [ ]:
plot_cumulative_returns(y_rev, '5-years momentum reversal')

In [ ]:
x, s= seas_prev_mth(snp_stocks_data, 2, 10)

In [ ]:
y_seas = total_return(snp_stocks_data, x, returns_df)

In [ ]:
plot_cumulative_returns(y_seas, 'Stock return > 10% last year')

In [ ]:
import helper
importlib.reload(helper)

In [ ]:
from helper import *

In [ ]:
x = value_strategy_bw(snp_stocks_data, 25, 15)

In [ ]:
y = total_return(snp_stocks_data, x, returns_df)

In [ ]:
plot_cumulative_returns(y, 'Stocks in 15%-25% quantile')

In [ ]:
x = calculate_m_month_volatility(snp_stocks_data)

In [ ]:
y_vol = bottom_p_percentage_stocks(x.T, 30)

In [ ]:
z = total_return(snp_stocks_data, y, returns_df)

In [ ]:
plot_cumulative_returns(z, 'Low volatility Stocks')

In [ ]:
sharpe_ratios = {}
strategies = ['Momentum', '12-1 momentum', 'Reversion', 'Seasonality', 'Value', 'Volatility']


In [ ]:
def cagr(data):   
    # Calculate the cumulative product of the returns
    end_value = 1
    for i in range(len(data)):
        end_value *= (1 + (data[i][1]/100))

    # Calculate the number of periods
    num_periods = len(data)
    
    # Calculate CAGR
    cagr = ((end_value) ** (1 / (num_periods/12))) - 1
    
    
    return cagr*100

In [ ]:
def sharpe_ratio(data, risk_free_rate=0):
    returns = np.array([x[1] for x in data])
    mean_return = np.mean(returns)
    std_dev = np.std(returns)
    
    start_date, end_date = data[0][0], data[-1][0]
    # Calculate the number of months
    num_months = (end_date.year - start_date.year) * 12 + end_date.month - start_date.month + 1
    
    # annualized_return = mean_return * 12  # Monthly return to annual return
    annualized_std_dev = std_dev * np.sqrt(12)  # Monthly std dev to annual std dev
    
    sharpe_ratio = (mean_return - risk_free_rate) / annualized_std_dev
    
    return sharpe_ratio

In [ ]:
(list(gains.items())[-m:])

In [ ]:
m = 12*1
b = cagr((list(gains.items())[-m:]))
print(b)

In [ ]:
(1.1824)**12

In [ ]:
((1+(gains.tail(12)/100)).cumprod().iloc[-1])-1

In [ ]:
x = pd.DataFrame(gains)

In [ ]:
calculate_yearly_cagr(x).loc['2023-12-31 00:00:00', 'Adj Close']

In [ ]:
price[-13]/price[-1]

In [ ]:
sharpe_ratio_df1

# Unit Test

In [ ]:
import unittest

In [ ]:
price # month end price of S&P500
gains # monthly return% of S&P500

In [ ]:
class TestNotebook(unittest.TestCase):
    def test_cagr(self):
        self.assertAlmostEqual(calculate_yearly_cagr(pd.DataFrame(gains)).loc['2023-12-31 00:00:00', 'Adj Close'], 24.25, 2)

    def test_sharpe(self):
        self.assertAlmostEqual(calculate_yearly_sharpe_ratio(pd.DataFrame(gains)).loc[2023, 'Adj Close'], 1.55, delta=0.05)

unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
sharpe_ratio_df